# Image Recognition Project - Structural Defect Recognition
---------------------------------------------------------------
## Data Visualisation

### Section Objectives
 - Complete the project objective and answer the business requirements by training the -

 --------------------------------------------------------------

## Import Libraries

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
from matplotlib.image import imread
from tensorflow.keras.preprocessing import image
import joblib

sns.set_style("whitegrid")

## Setting Working Directory

In [2]:
cwd= os.getcwd()

In [4]:
os.chdir('/workspaces/ML_Project_Image_Recognition')
print("You set a new current directory")

You set a new current directory


In [5]:
work_dir = os.getcwd()
work_dir

'/workspaces/ML_Project_Image_Recognition'

## Setting Input and Output Directories

### Input:

In [10]:
base_path = 'inputs/cracks_dataset_new' 

train_path = os.path.join(base_path, 'train')
val_path = os.path.join(base_path, 'val')
test_path = os.path.join(base_path, 'test')

datasets = os.listdir(train_path)
print("Detected datasets in training set:", datasets)


Detected datasets in training set: ['Decks', 'Pavements', 'Walls']


### Output: